In [10]:
from keras.layers import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM, GRU
from keras.models import Sequential 
import numpy as np

In [23]:
inputFile = open("the_agile_samurai.txt", 'rb')
lines = []
for line in inputFile:
    line = line.strip().lower()
    line = line.decode("utf8", "ignore")
    if len(line) == 0:
        continue
    lines.append(line)
inputFile.close()
text = " ".join(lines)

In [24]:
chars = set([c for c in text])
nb_chars = len(chars)
char2index = dict((c, i) for i, c in enumerate(chars))
index2char = dict((i, c) for i, c in enumerate(chars))

In [25]:
SEQLEN = 60
STEP = 1
input_chars = []
label_chars = []
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i:i + SEQLEN])
    label_chars.append(text[i + SEQLEN])

print(input_chars[0])
print(label_chars[0])

agile in a nutshell what would it take to deliver something 
o


In [26]:
X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1

In [32]:
HIDDEN_SIZE = 256
BATCH_SIZE = 512
ITERATIONS = 5
EPOCHS_PER_ITERATION = 1
PREDS_PER_EPOCH = 20

In [33]:
input_shape = (int(SEQLEN), int(nb_chars))

In [34]:
model = Sequential()
#model.add(LSTM(HIDDEN_SIZE, return_sequences=False, input_shape=(SEQLEN, nb_chars), unroll=True))
model.add(GRU(HIDDEN_SIZE, return_sequences=True, input_shape=input_shape))
model.add(GRU(int(HIDDEN_SIZE/2)))
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(nb_chars, activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [35]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_10 (GRU)                 (None, 60, 256)           247296    
_________________________________________________________________
gru_11 (GRU)                 (None, 128)               147840    
_________________________________________________________________
dense_6 (Dense)              (None, 100)               12900     
_________________________________________________________________
dropout_4 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 65)                6565      
Total params: 414,601
Trainable params: 414,601
Non-trainable params: 0
_________________________________________________________________


In [36]:
for iteration in range(ITERATIONS):
    print("=" * 50)
    print("Iteration #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=EPOCHS_PER_ITERATION)
    
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]
    print("Generating from seed: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(PREDS_PER_EPOCH):
        Xtest = np.zeros((1, SEQLEN, nb_chars))
        for i, ch in enumerate(test_chars):
            Xtest[0, i, char2index[ch]] = 1
        pred = model.predict(Xtest, verbose=0)[0]        
        ypred = index2char[np.argmax(pred)]
        print(ypred, end="")
        # move forward with test_chars + ypred
        test_chars = test_chars[1:] + ypred


Iteration #: 0
Epoch 1/1
255855/255855 [==============================] - 72s - loss: 3.1557 - acc: 0.1767    
Generating from seed: ed and focused. setting up a visual workspace flight status 
ed and focused. setting up a visual workspace flight status te e oe e oe e oe e ==================================================
Iteration #: 1
Epoch 1/1
255855/255855 [==============================] - 71s - loss: 2.5163 - acc: 0.3009    
Generating from seed: deeply interested in understanding what the customer needs a
deeply interested in understanding what the customer needs and the the the the t==================================================
Iteration #: 2
Epoch 1/1
255855/255855 [==============================] - 71s - loss: 2.2000 - acc: 0.3753    
Generating from seed: on all the risk and uncertainty that comes with large projec
on all the risk and uncertainty that comes with large project the the the the th==================================================
Iteration #: 3
Epoch 1/1


KeyboardInterrupt: 